<a href="https://colab.research.google.com/github/lenahdtr/fallstudie_lkws/blob/main/Stufe2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie- LKW Zuteilung - Stufe 2


In [ ]:
! pip install -q pyscipopt

In [ ]:
import pandas as pd
from pyscipopt import Model, quicksum

# Daten laden

*Daten aus den Tabellen, die hard codiert werden:*

In [ ]:
routes_data = [
    {"route_id": "t-4", "distance_total": 250.0, "distance_toll": 150.0, "starttime": 0.28125, "endtime": 0.71875},
    {"route_id": "t-5", "distance_total": 250.0, "distance_toll": 150.0, "starttime": 0.2708333333333333, "endtime": 0.7083333333333334},
    {"route_id": "t-6", "distance_total": 250.0, "distance_toll": 150.0, "starttime": 0.25, "endtime": 0.6875},

    {"route_id": "s-1", "distance_total": 120.0, "distance_toll": 32.0, "starttime": 0.22916666666666666, "endtime": 0.6458333333333334},
    {"route_id": "s-2", "distance_total": 120.0, "distance_toll": 32.0, "starttime": 0.25, "endtime": 0.6666666666666666},
    {"route_id": "s-3", "distance_total": 120.0, "distance_toll": 32.0, "starttime": 0.375, "endtime": 0.6666666666666666},
    {"route_id": "s-4", "distance_total": 120.0, "distance_toll": 32.0, "starttime": 0.2708333333333333, "endtime": 0.6875},

    {"route_id": "w1", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.22916666666666666, "endtime": 0.6458333333333334},
    {"route_id": "w2", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.3333333333333333, "endtime": 0.75},
    {"route_id": "w3", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.28125, "endtime": 0.6979166666666666},
    {"route_id": "w4", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.25, "endtime": 0.6666666666666666},
    {"route_id": "w5", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.2916666666666667, "endtime": 0.7083333333333334},
    {"route_id": "w6", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.22916666666666666, "endtime": 0.6458333333333334},
    {"route_id": "w7", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.3020833333333333, "endtime": 0.71875},

    {"route_id": "r1", "distance_total": 285.0, "distance_toll": 259.0, "starttime": 0.75, "endtime": 0.9375},
    {"route_id": "r2", "distance_total": 250.0, "distance_toll": 220.0, "starttime": 0.6875, "endtime": 0.90625},
    {"route_id": "r3", "distance_total": 235.0, "distance_toll": 219.0, "starttime": 0.7395833333333334, "endtime": 0.8958333333333334},

    {"route_id": "h3", "distance_total": 180.0, "distance_toll": 160.0, "starttime": 0.78125, "endtime": 0.9479166666666666},
    {"route_id": "h4", "distance_total": 180.0, "distance_toll": 160.0, "starttime": 0.7708333333333334, "endtime": 0.9375},

    {"route_id": "k1", "distance_total": 275.0, "distance_toll": 235.0, "starttime": 0.6875, "endtime": 0.9375},
]
routes_df = pd.DataFrame(routes_data)

# Initialisierung des Modells

In [ ]:
model = Model("Stufe2_Diesel-Elektro_Optimierung")

# 1. Indexmengen

In [ ]:
ROUTEN = routes_df.index.tolist()   # Menge aller Touren

In [ ]:
FUHRPARK = range(1, len(ROUTEN)+1)  # Alle geleasten LKWs im Fuhrpark

In [ ]:
FTypen = ["Diesel", "E400", "E600"]                      # Verfügbare LKW-Typen

# 2. Parameter

LKW-Kosten

In [ ]:
C_CAPEX = {"Diesel": 24000.0, "E400": 50000.0, "E600": 60000.0}
C_OPEX = {"Diesel": 6000.0, "E400": 5000.0, "E600": 6000.0}
C_STEUER = {"Diesel": 556.0, "E400": 0.0, "E600": 0.0}
THG = {"Diesel": 0.0, "E400": 1000.0, "E600": 1000.0}
VERBRAUCH = {"Diesel": 26.0/100, "E400": 105.0/100, "E600": 110.0/100}
BATTERIE = {"E400": 414.0, "E600": 621.0}

Preise

In [ ]:
P_DIESEL = 1.60                                  # Kraftstoffpreis
P_STROM = 0.25                                   # Stromkosten pro kwH

Tour

In [ ]:
# Mautgebühr
P_MAUT = 0.34

#Gesamte Fahrstrecke der Tour r
D_GES = {r: routes_df.loc[r, 'distance_total'] for r in ROUTEN}  #Wir schauen in jeder Spalte Gesamtdistanz in der Tabelle von den Routen und nehmen das für jede ROute raus

#Mautpflichtige STrecke der Tour r
D_MAUT = {r: routes_df.loc[r, 'distance_toll'] for r in ROUTEN}  #siehe oben

Zeit

In [ ]:
#Startzeitpunkt der Tour r
START = {r: routes_df.loc[r, 'starttime'] * 1440 for r in ROUTEN}    #siehe oben: 1440, weil es 1440 Minuten am Tag

#Endzeitpunkt der Tour r
END = {r: routes_df.loc[r, 'endtime'] * 1440 for r in ROUTEN}

#Einsatztage pro Jahr
N_TAGE = 260

# 3. Entscheidungsvariablen

In [ ]:
# a[lkw, tour]: 1, wenn LKW f die Tour r fährt
a = {}
for f in FUHRPARK:
    for r in ROUTEN:
        a[f, r] = model.addVar(vtype="B", name=f"fahrt_{f}_{r}")

In [ ]:
# y[lkw, typ]: 1, wenn LKW f als Typ t gekauft wird
y = {}
for f in FUHRPARK:
    for t in FTypen:
        y[f, t] = model.addVar(vtype="B", name=f"kauf_{f}_{t}")


In [ ]:
# x[typ]: Anzahl der LKWs f pro Typ t
x = {}
for t in FTypen:
    x[t] = model.addVar(vtype="I", name=f"anzahl_{t}")

In [ ]:
# Bedeutung: LKW f vom Typ t fährt Route r (z[f,r,t] ersetzt das Produkt (a * y))
z = {(f, r, t): model.addVar(vtype="B") for f in FUHRPARK for r in ROUTEN for t in FTypen}

# 4. Zielfunktion

In [ ]:
# TEIL A: FIXKOSTEN (Was uns die LKWs pro Jahr kosten, egal wie viel sie fahren)
# Wir summieren für jeden Typ (Diesel, E400, E600):
# (Anzahl LKWs) * (Leasing + Wartung + Steuer - THG-Bonus)
fix_kosten = quicksum(x[t] * (C_CAPEX[t] + C_OPEX[t] + C_STEUER[t] - THG[t]) for t in FTypen)

# TEIL B: VARIABLE KOSTEN (Sprit/Strom & Maut pro Tag)
# Wir nutzen z[f, r, t]
variable_kosten_pro_tag = 0

for f in FUHRPARK:
    # 1. Fall: Der LKW auf Platz 'f' ist ein Diesel-LKW
    # Wir berechnen: (Gefahrene km * Dieselpreis) + (Maut-km * Mautpreis)
    kosten_diesel = quicksum(z[f, r, "Diesel"] * (D_GES[r] * VERBRAUCH["Diesel"] * P_DIESEL + D_MAUT[r] * P_MAUT)
                             for r in ROUTEN)  #r in ROUTEN machen wir nur weil ein LKW ja auch mehrere Routen fahren kann

    # 2. Fall: Der LKW auf Platz 'f' ist Elektro (E400 oder E600)
    # Wir berechnen: (Gefahrene km * Strompreis)
    kosten_elektro_400 = quicksum(z[f, r, "E400"] * (D_GES[r] * VERBRAUCH["E400"] * P_STROM)
                                  for r in ROUTEN)
    kosten_elektro_600 = quicksum(z[f, r, "E600"] * (D_GES[r] * VERBRAUCH["E600"] * P_STROM)
                                  for r in ROUTEN)

  # Wir addieren die Fälle einfach auf. Da ein LKW nur EINEN Typ haben kann (NB 4),
    variable_kosten_pro_tag += kosten_diesel + kosten_elektro_400 + kosten_elektro_600

# GESAMTZIEL: Fixkosten + (260 Tage * tägliche Kosten)
model.setObjective(fix_kosten + (N_TAGE * variable_kosten_pro_tag), "minimize")

# 5. Nebenbedingungen

In [ ]:
# LINEARE NEBENBEDINGUNGEN
for f in FUHRPARK:
    for r in ROUTEN:
        for t in FTypen:
            # Diese 3 Regeln erzwingen z = a * y (Linearisierung)
            model.addCons(z[f, r, t] <= a[f, r])      # z kann nur 1 sein, wenn Tour zugewiesen
            model.addCons(z[f, r, t] <= y[f, t])      # z kann nur 1 sein, wenn Typ zugewiesen
            model.addCons(z[f, r, t] >= a[f, r] + y[f, t] - 1) # Wenn beides 1, muss z auch 1 sein

In [ ]:
# NB 1: Tourenabdeckung -> Jede Tour muss genau einmal gefahren werden
for r in ROUTEN:
    model.addCons(quicksum(a[f, r] for f in FUHRPARK) == 1)

In [ ]:
# NB 2: Zeit-Check (Keine Tour-Überlappung pro LKW)
for f in FUHRPARK: #für jeden LKW im FUhrpark
    for r in ROUTEN:  #(+nächste Zeile:) schaue mir jedes Paar von den Routen an also r=Rout 1 und k = Route2
        for k in ROUTEN:
          # Dass jedes Paar nur einmal geprüft wird
          if r < k and START[k] < END[r] and START[r] < END[k]:       #Route k startet bevor Route r endet UND Route r startet bevor Route k endet
                    model.addCons(a[f, r] + a[f, k] <= 1)

In [ ]:
# NB 3: Typ-Eindeutigkeit -> Ein LKW f muss genau einem Typ entsprechen
for f in FUHRPARK:
    model.addCons(quicksum(y[f, t] for t in FTypen) == 1)  #wenn wir das nicht haben, kauft das Programm "Geister-LKWs" weil die nichts kosten weil wir wollen kosten minimieren (Unsere Kosten an den Typ gekoppelt, wenn LKW keinen Typ hat, hat er auch keine Kosten)

In [ ]:
# NB 4: Kauf-Kopplung (Fahren nur bei Kauf) -> LKW (ein bisschen redundant, weil jeder LKW einem Typ zugewiesen wird, aber drinne lassen, damit Schlupflöcher weg)
for f in FUHRPARK:
        model.addCons(quicksum(a[f, r] for r in ROUTEN) <= len(ROUTEN) * quicksum(y[f, t] for t in FTypen))  # Wenn rechts d.h. y=0 ist dann hat LKW nicht Typ t, aber dann muss links a (=LkW fährt TOUR) auch 0 sein also LKW fährt nicht weil <=

In [ ]:
# NB 5: Flottenberechnung -> Fuhrpark ist genau so groß wie alle beschaffenen LKWs
for t in FTypen:
    model.addCons(x[t] == quicksum(y[f, t] for f in FUHRPARK))      #x =Anzahl LKWs ist genauso groß wie die SUmme aus allen Fahrzeugen von den Typen

In [ ]:
# NB 6: Summe der Verbräuche der Touren von E-LKW f darf Batteriekapazität nicht übersteigen
for f in FUHRPARK:
  for t in ["E400", "E600"]:
    model.addCons(quicksum(z[f, r, t] * D_GES[r] * VERBRAUCH[t] for r in ROUTEN) <= BATTERIE[t])
    # Der Gesamtverbrauch aller Touren eines LKW f muss <= Batteriekapazität des gewählten Typs sein

Modell optimieren

In [ ]:
#model.setRealParam("limits/time", 600) # Gib ihm 10 Minuten Zeit
model.optimize()

Ausgabe

In [ ]:
# Suche nach der besten ZULÄSSIGEN Lösung
if model.getNSols() > 0:
    print("\n" + "="*50)
    print(f"ERGEBNIS: {model.getObjVal():,.2f} € Gesamtkosten/Jahr")
    print("="*50)

    # 1. ANZAHL UND TYP DER LKWS
    print("\n--- FLOTTENZUSAMMENSETZUNG ---")
    for t in FTypen:
        anzahl = int(model.getVal(x[t]))
        if anzahl > 0:
            # Wir nutzen die Parameter aus deinem Modell für die Anzeige
            print(f"➤ {t:8}: {anzahl} Fahrzeug(e)")

    # 2. ZUORDNUNG DER TOUREN
    print("\n--- EINSATZPLAN PRO LKW-SLOT ---")
    gefundene_lkws = 0
    for f in FUHRPARK:
        # Prüfen, welche Touren diesem Slot f zugeordnet sind
        tours_for_f = [routes_df.loc[r, 'route_id'] for r in ROUTEN if model.getVal(a[f, r]) > 0.5]

        if tours_for_f:
            gefundene_lkws += 1
            # Herausfinden, welcher Typ für diesen Slot gewählt wurde
            lkw_typ = "Unbekannt"
            for t in FTypen:
                if model.getVal(y[f, t]) > 0.5:
                    lkw_typ = t
                    break

            # Schöne Auflistung der Touren
            tour_liste = ", ".join(tours_for_f)
            print(f"[{lkw_typ:6}]: fährt Tour(en) -> {tour_liste}")

    if gefundene_lkws == 0:
        print("Keine aktiven LKW-Zuweisungen gefunden.")

else:
    print("\n" + "!"*50)
    print("Der Solver sagt: Es gibt mathematisch KEINE Lösung.")
    print("Mögliche Gründe:")
    print("- Eine Tour ist zu lang für die Batteriekapazität (NB 6)")
    print("- Zeit-Überschneidungen verhindern die Zuweisung (NB 2)")
    print("- Die Kopplung zwischen Fahrt und Kauf ist fehlerhaft (NB 3)")
    print("!"*50)

# Status-Check für die Dokumentation
print(f"\nAbschluss-Status des Solvers: {model.getStatus()}")


ERGEBNIS: 1,072,596.85 € Gesamtkosten/Jahr

--- FLOTTENZUSAMMENSETZUNG ---
➤ Diesel  : 14 Fahrzeug(e)
➤ E400    : 3 Fahrzeug(e)
➤ E600    : 3 Fahrzeug(e)

--- EINSATZPLAN PRO LKW-SLOT ---
[Diesel]: fährt Tour(en) -> w4
[Diesel]: fährt Tour(en) -> s-4
[E400  ]: fährt Tour(en) -> s-1, h4
[Diesel]: fährt Tour(en) -> w5
[E600  ]: fährt Tour(en) -> t-6, k1
[Diesel]: fährt Tour(en) -> w2
[E600  ]: fährt Tour(en) -> t-4, h3
[E400  ]: fährt Tour(en) -> s-3, r2
[Diesel]: fährt Tour(en) -> w3
[Diesel]: fährt Tour(en) -> w1
[E600  ]: fährt Tour(en) -> t-5, r1
[Diesel]: fährt Tour(en) -> w7
[Diesel]: fährt Tour(en) -> w6
[E400  ]: fährt Tour(en) -> s-2, r3

Abschluss-Status des Solvers: optimal
